In [1]:
import emission.storage.timeseries.timequery as estt
import emission.storage.timeseries.abstract_timeseries as esta

In [5]:
import emission.core.get_database as edb
import bson.objectid as boi
from uuid import UUID

In [3]:
edb.get_timeseries_db().find_one({"_id": boi.ObjectId('57e44d88f6858f7387263233')})

{u'_id': ObjectId('57e44d88f6858f7387263233'),
 u'data': {u'accuracy': 33.941124,
  u'altitude': 0,
  u'elapsedRealtimeNanos': 610126618689207L,
  u'filter': u'time',
  u'fmt_time': u'2016-06-22T20:09:17-07:00',
  u'heading': 0,
  u'latitude': 37.86835491657257,
  u'loc': {u'coordinates': [-122.25289285182953, 37.86835491657257],
   u'type': u'Point'},
  u'local_dt': {u'day': 22,
   u'hour': 20,
   u'minute': 9,
   u'month': 6,
   u'second': 17,
   u'timezone': u'America/Los_Angeles',
   u'weekday': 2,
   u'year': 2016},
  u'longitude': -122.25289285182953,
  u'sensed_speed': 0,
  u'ts': 1466651357},
 u'metadata': {u'key': u'background/filtered_location',
  u'platform': u'android',
  u'read_ts': 0,
  u'time_zone': u'America/Los_Angeles',
  u'type': u'sensor-data',
  u'write_fmt_time': u'2016-06-22T21:43:36.201000-07:00',
  u'write_local_dt': {u'day': 22,
   u'hour': 21,
   u'minute': 43,
   u'month': 6,
   u'second': 36,
   u'timezone': u'America/Los_Angeles',
   u'weekday': 2,
   u'ye

In [6]:
tq = estt.TimeQuery("metadata.write_ts", 1466657016.201 - 100, 1466657016.201 + 100)
loc_df = esta.TimeSeries.get_time_series(UUID('96993ce8-e550-4535-aa28-836804596d92')).get_data_df("background/filtered_location", tq)

In [7]:
len(loc_df)

10

In [8]:
loc_df.ts.diff().tolist()

[nan,
 17.506999969482422,
 14.493000030517578,
 16.608999967575073,
 15.391000032424927,
 -5633.0,
 5660.6779999732971,
 30.950000047683716,
 31.240000009536743,
 31.131999969482422]

In [9]:
to_del = loc_df[loc_df.ts.diff() < 0][["fmt_time", "_id"]]
print to_del.index, len(to_del)
to_del

Int64Index([5], dtype='int64') 1


,fmt_time,_id
5,2016-06-22T20:09:17-07:00,57e44d88f6858f7387263233


In [11]:
len(loc_df.drop(to_del.index.tolist()))

9

In [14]:
out_of_order_id_list = to_del["_id"].tolist()
edb.get_timeseries_db().remove({"_id": {"$in": out_of_order_id_list}})

{u'n': 1, u'ok': 1}